## Installation

In [ ]:
%%capture
!pip install unsloth==2025.1.6 evaluate==0.4.3 rouge_score==0.1.2

In case any issues arise with Unsloth check out this Python Notebook for updates on installation: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb

## Global Variables

In [ ]:
dataset_name = (
    input(
        "Enter the name of the generated dataset during Lesson 3. Hit enter to default to our cached generated dataset."
    )
    or "pauliusztin/second_brain_course_summarization_task"
)
print(f"{dataset_name=}")
model_name = (
    input(
        "Enter the name of fine-tuned LLM during Lesson 4. Hit enter to default to our fine-tuned LLM."
    )
    or "pauliusztin/Meta-Llama-3.1-8B-Instruct-Second-Brain-Summarization"
)
print(f"{model_name=}")

Enter the name of the generated dataset during Lesson 3. Hit enter to default to our cached generated dataset.
Enter the name of fine-tuned LLM during Lesson 4. Hit enter to default to our fine-tuned LLM.


## Load Fine-tuned LLM

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=8192,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


KeyboardInterrupt: 

## Prepare Input Samples

In [ ]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a helpful assistant specialized in summarizing documents. Generate a concise TL;DR summary in markdown format having a maximum of 512 characters of the key findings from the provided documents, highlighting the most significant insights

### Input:
{}

### Response:
{}"""

In [ ]:
dataset = load_dataset(dataset_name, split="test")

In [ ]:
dataset[0]["instruction"][:1000]

In [ ]:
dataset[0]["answer"][:1000]

## Inference

In [ ]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


def generate_text(
    instruction, streaming: bool = True, trim_input_message: bool = False
):
    message = alpaca_prompt.format(
        instruction,
        "",  # output - leave this blank for generation!
    )
    inputs = tokenizer([message], return_tensors="pt").to("cuda")

    if streaming:
        return model.generate(
            **inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True
        )
    else:
        output_tokens = model.generate(**inputs, max_new_tokens=256, use_cache=True)
        output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

        if trim_input_message:
            return output[len(message) :]
        else:
            return output

In [ ]:
_ = generate_text(dataset[0]["instruction"], streaming=True)

In [ ]:
generate_text(dataset[0]["instruction"], streaming=False)


And we're done! If you have any questions on Unsloth, they have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join their Discord!

Also, you can visit their docs to learn about their supported [models](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
4. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>